In [11]:
import numpy as np
import pandas as pd
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

accounts = pd.read_csv("csv_files/accounts.csv", encoding="utf-8", dtype={"phone": str})
parties = pd.read_csv("csv_files/parties.csv", encoding="utf-8", parse_dates=["begin_time", "end_time", "start_time", "updated_at"])
party_user = pd.read_csv("csv_files/party_user.csv", encoding="utf-8", parse_dates=["created_at", "updated_at"])
users = pd.read_csv("csv_files/users.csv", encoding="utf-8")
user_sessions = pd.read_csv("csv_files/user_sessions.csv", encoding="utf-8")
party_played_items = pd.read_csv("csv_files/party_played_items.csv", encoding="utf-8", parse_dates=["played_at", "updated_at"])
messages = pd.read_csv("csv_files/messages.csv", encoding="utf-8", parse_dates=["created_at"])

In [2]:

dff = pd.merge(party_user[["party_id", "user_id", "created_at", "role", "updated_at"]], parties[["id", "begin_time", "end_time", "updated_at", "creator_id"]], how="left", left_on="party_id", right_on="id")
del dff["id"]
dff.columns = ['party_id', 'user_id', 'user_party_created_at','role','user_party_updated_at', 'party_begin_time', 'party_end_time', 'party_updated_at', 'creator_id']
display(dff.head())

,party_id,user_id,user_party_created_at,role,user_party_updated_at,party_begin_time,party_end_time,party_updated_at,creator_id
0,e466a9f2-5318-11ec-a8a2-8daeb074a1ae,73e6789c-41ca-11ec-a8b6-a4c0148ad650,2021-12-02 02:38:11.795,host,2021-12-02 02:38:44.193,2021-12-02 02:38:44.193,2021-12-02 02:46:51.076,2021-12-02 02:46:51.076,73e6789c-41ca-11ec-a8b6-a4c0148ad650
1,7d7fbcc4-4c92-11ec-8907-98f59b9e3b1b,ee248f30-4bd5-11ec-9b73-dd21c2b4b8f3,2021-11-23 19:20:59.615,host,2021-11-23 19:21:29.959,2021-11-23 19:23:30.736,NaT,2021-11-23 19:23:30.736,ee248f30-4bd5-11ec-9b73-dd21c2b4b8f3
2,7d7fbcc4-4c92-11ec-8907-98f59b9e3b1b,5e64bb62-4be0-11ec-aad5-eccb9bc4f187,2021-11-23 19:21:18.843,guest,2021-11-23 19:23:30.196,2021-11-23 19:23:30.736,NaT,2021-11-23 19:23:30.736,ee248f30-4bd5-11ec-9b73-dd21c2b4b8f3
3,61640b36-354e-11ec-b19b-1a6c70deb5be,fcd85ac4-354c-11ec-8a67-a069389d8a76,2021-10-25 04:45:29.936,host,2021-10-25 04:45:57.913,2021-10-25 04:56:42.993,NaT,2021-10-25 04:56:42.994,fcd85ac4-354c-11ec-8a67-a069389d8a76
4,579ab14e-4071-11ec-ad47-a273009342b9,2350f594-3287-11ec-9bc4-d0dbea1a278a,2021-11-08 08:53:28.692,host,2021-11-30 08:51:48.542,2021-11-30 08:51:48.542,2021-11-30 08:55:47.002,2021-11-30 08:56:12.812,2350f594-3287-11ec-9bc4-d0dbea1a278a


In [3]:
dff_ = pd.merge(dff, users[["id", "account_id", "username"]], how="left", left_on="user_id", right_on="id")
del dff_['id']
dff_ = pd.merge(dff_, users[["id", "account_id", "username"]], how="left", left_on="creator_id", right_on="id", suffixes=('_pu', '_creator'))

In [4]:
def fill_begin_time(row):
    if row["party_begin_time"] is pd.NaT:
        return row["user_party_created_at"]
    else:
        return row["party_begin_time"]

def fill_end_time(row):
    if row["party_end_time"] is pd.NaT:
        return row["user_party_updated_at"]
    else:
        return row["party_end_time"]

dff_["party_begin_time"] = dff_.apply(fill_begin_time, axis=1)
dff_["party_end_time"] = dff_.apply(fill_end_time, axis=1)

In [5]:

dff_["party_start_date"] = dff_["party_begin_time"].dt.date
# display(dff_.head(20))

In [6]:
df_ = pd.merge(
    dff_,
    accounts[["id", "phone"]],
    how="left",
    left_on="account_id_pu",
    right_on="id",
)
df_.to_csv("df_.csv", encoding="utf-8")